In [1]:
%matplotlib inline

In [2]:
import os
import sys

In [3]:
print sys.version

2.7.14 (default, Jan 17 2018, 15:13:18) 
[GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.39.2)]


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [5]:
import xgboost as xgb
import pymatgen as mg
import datetime as dt

/usr/local/lib/python2.7/site-packages/pymatgen/__init__.py:87: UserWarning: 
Pymatgen will drop Py2k support from v2019.1.1. Pls consult the documentation
at https://www.pymatgen.org for more details.
  at https://www.pymatgen.org for more details.""")


In [6]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [7]:
from utils import rmsle

### Load data

In [8]:
DATA_DIR = './data'

In [9]:
train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'),
                    names=['id', 'spacegroup', 'natoms', 'al',
                           'ga', 'in', 'a', 'b', 'c',
                           'alpha', 'beta',
                           'gamma', 'E0',
                           'bandgap'],
                    header=0,
                    sep=',')
test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'),
                    names=['id', 'spacegroup', 'natoms', 'al',
                           'ga', 'in', 'a', 'b', 'c',
                           'alpha', 'beta',
                           'gamma'],
                    header=0,
                    sep=',')

In [10]:
train_ext = pd.read_csv(os.path.join(DATA_DIR, 'train_ext.csv'),
                    header=0,
                    sep=',')
test_ext = pd.read_csv(os.path.join(DATA_DIR, 'test_ext.csv'),
                    header=0,
                    sep=',')

In [11]:
train = train.merge(train_ext, on='id')
test = test.merge(test_ext, on='id')

#### Add the spacegroup_natoms category:

In [12]:
train['spacegroup_natoms'] = train['spacegroup'].astype(str) +\
    '_' + train['natoms'].astype(int).astype(str)
test['spacegroup_natoms'] = test['spacegroup'].astype(str) +\
    '_' + test['natoms'].astype(int).astype(str)

#### Add cell volume and calculate atomic and mass density:

In [13]:
train['cellvol'] = train.apply(lambda r: mg.Lattice.from_parameters(r['a'], r['b'], r['c'],
                                         r['alpha'], r['beta'], r['gamma']).volume,
                               axis=1)
test['cellvol'] = test.apply(lambda r: mg.Lattice.from_parameters(r['a'], r['b'], r['c'],
                                       r['alpha'], r['beta'], r['gamma']).volume,
                             axis=1)

In [14]:
train['atom_density'] = train['natoms'] / train['cellvol']
test['atom_density'] = test['natoms'] / test['cellvol']

In [15]:
train['mass_density'] = train['avg_mass'] / train['cellvol']
test['mass_density'] = test['avg_mass'] / test['cellvol']

#### Convert angles to radians:

In [16]:
train['alpha_r'] = np.radians(train['alpha'])
train['beta_r'] = np.radians(train['beta'])
train['gamma_r'] = np.radians(train['gamma'])
test['alpha_r'] = np.radians(test['alpha'])
test['beta_r'] = np.radians(test['beta'])
test['gamma_r'] = np.radians(test['gamma'])

#### Check O fraction = 0.6:

In [17]:
train['o_fraction'] = train['o_cnt'] / train['natoms']
test['o_fraction'] = test['o_cnt'] / test['natoms']

In [18]:
assert (train['o_fraction'] == 0.6).all()
assert (test['o_fraction'] == 0.6).all()

### Build train and test data sets

In [19]:
X_train = train.drop(['id', 'natoms', 'spacegroup',
                      'alpha', 'beta', 'gamma',
                      'ga', 'o_cnt', 'cellvol', 'o_fraction', 'avg_mass',
                      'bandgap', 'E0'], axis=1)
X_test = test.drop(['id', 'natoms', 'spacegroup',
                    'alpha', 'beta', 'gamma',
                    'ga', 'o_cnt', 'cellvol', 'o_fraction', 'avg_mass'], axis=1)

In [20]:
# Use log1p of energies to correct for skew
y_bg_train = train['bandgap']
y_e0_train = np.log1p(train['E0'])

In [21]:
# One-hot encode spacegroup_natoms
X_train = pd.concat([X_train.drop('spacegroup_natoms', axis=1),
                    pd.get_dummies(X_train['spacegroup_natoms'])], axis=1)
X_test = pd.concat([X_test.drop('spacegroup_natoms', axis=1),
                    pd.get_dummies(X_test['spacegroup_natoms'])], axis=1)

### Build Sklearn Model with XGBRegressor

In [38]:
param = {'learning_rate': 0.10,      # Step size shrinkage used in update (Learning rate)
         'reg_alpha': 0.01,          # L1 regularization term on weights
         'n_estimators': 60,
         'max_depth': 5,
         'subsample': 1,
         'colsample_bytree': 0.90,
         'colsample_bylevel': 0.90,
         'silent': True,
         'random_state': 42,
         'objective': 'reg:linear'}

In [39]:
# Estimator pipeline
est = Pipeline([
    ('scaler', StandardScaler()),
    ('xgbreg', XGBRegressor(**param)),
])

In [40]:
est

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.9, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=60,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
       reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

In [41]:
# Perform grid search for bandgap model
t_md = [3, 4, 5, 6, 7]
t_lr = [0.01, 0.02, 0.05, 0.10, 0.20, 0.3]
t_ne = range(50, 500, 50)
gridsearch_bg_0 = GridSearchCV(est,
                              {'xgbreg__max_depth': t_md,
                               'xgbreg__learning_rate': t_lr,
                               'xgbreg__n_estimators': t_ne,
                              },
                              cv=5,
                              n_jobs=4,
                              scoring='neg_mean_squared_error',
                              return_train_score=False,
                              verbose=True)

In [42]:
gridsearch_bg_0.fit(X_train, y_bg_train)

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


[Parallel(n_jobs=4)]: Done  61 tasks      | elapsed:    8.7s
[Parallel(n_jobs=4)]: Done 211 tasks      | elapsed:   52.6s
[Parallel(n_jobs=4)]: Done 461 tasks      | elapsed:  2.2min
[Parallel(n_jobs=4)]: Done 811 tasks      | elapsed:  3.7min
[Parallel(n_jobs=4)]: Done 1261 tasks      | elapsed:  5.6min
[Parallel(n_jobs=4)]: Done 1350 out of 1350 | elapsed:  6.0min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.9, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_...    reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'xgbreg__n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450], 'xgbreg__max_depth': [3, 4, 5, 6, 7], 'xgbreg__learning_rate': [0.01, 0.02, 0.05, 0.1, 0.2, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='neg_mean_squared_error', verbose=True)

In [43]:
gridsearch_bg_0.best_params_

{'xgbreg__learning_rate': 0.05,
 'xgbreg__max_depth': 3,
 'xgbreg__n_estimators': 250}

In [50]:
-gridsearch_bg_0.best_score_

0.047868832486723044

In [51]:
# Check best regularization
t_ra = [0.1, 0.05, 0.01, 0.005, 0.001]
gridsearch_bg_1 = GridSearchCV(est,
                              {'xgbreg__reg_alpha': t_ra,
                              },
                              cv=5,
                              n_jobs=4,
                              scoring='neg_mean_squared_error',
                              return_train_score=False,
                              verbose=True)

In [52]:
gridsearch_bg_1.fit(X_train, y_bg_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    1.7s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.9, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_...    reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'xgbreg__reg_alpha': [0.1, 0.05, 0.01, 0.005, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='neg_mean_squared_error', verbose=True)

In [53]:
gridsearch_bg_1.best_params_

{'xgbreg__reg_alpha': 0.01}

In [54]:
-gridsearch_bg_1.best_score_

0.05032338635026175

In [45]:
# Perform grid search for formation energy model
t_md = [3, 4, 5, 6, 7]
t_lr = [0.01, 0.02, 0.05, 0.10, 0.20, 0.3]
t_ne = range(50, 500, 50)
gridsearch_e0_0 = GridSearchCV(est,
                              {'xgbreg__max_depth': t_md,
                               'xgbreg__learning_rate': t_lr,
                               'xgbreg__n_estimators': t_ne,
                              },
                              cv=5,
                              n_jobs=4,
                              scoring='neg_mean_squared_error',
                              return_train_score=False,
                              verbose=True)

In [46]:
gridsearch_e0_0.fit(X_train, y_e0_train)

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


[Parallel(n_jobs=4)]: Done  61 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done 211 tasks      | elapsed:   49.5s
[Parallel(n_jobs=4)]: Done 461 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done 811 tasks      | elapsed:  3.3min
[Parallel(n_jobs=4)]: Done 1261 tasks      | elapsed:  5.0min
[Parallel(n_jobs=4)]: Done 1350 out of 1350 | elapsed:  5.2min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.9, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_...    reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'xgbreg__n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450], 'xgbreg__max_depth': [3, 4, 5, 6, 7], 'xgbreg__learning_rate': [0.01, 0.02, 0.05, 0.1, 0.2, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='neg_mean_squared_error', verbose=True)

In [47]:
gridsearch_e0_0.best_params_

{'xgbreg__learning_rate': 0.05,
 'xgbreg__max_depth': 4,
 'xgbreg__n_estimators': 150}

In [48]:
-gridsearch_e0_0.best_score_

0.0011021747873143457

In [59]:
# Check best regularization
t_ra = [0.2, 0.1, 0.05, 0.01, 0.005, 0.001]
gridsearch_e0_1 = GridSearchCV(est,
                              {'xgbreg__reg_alpha': t_ra,
                              },
                              cv=5,
                              n_jobs=4,
                              scoring='neg_mean_squared_error',
                              return_train_score=False,
                              verbose=True)

In [60]:
gridsearch_e0_1.fit(X_train, y_e0_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    1.9s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.9, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_...    reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'xgbreg__reg_alpha': [0.2, 0.1, 0.05, 0.01, 0.005, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='neg_mean_squared_error', verbose=True)

In [61]:
gridsearch_e0_1.best_params_

{'xgbreg__reg_alpha': 0.1}

In [62]:
-gridsearch_e0_1.best_score_

0.001104495414100288

### Fit models using optimized hyperparameters

In [64]:
pa_bg = {'learning_rate': 0.05,      # Step size shrinkage used in update (Learning rate)
         'reg_alpha': 0.01,          # L1 regularization term on weights
         'n_estimators': 250,
         'max_depth': 3,
         'subsample': 1,
         'colsample_bytree': 0.90,
         'colsample_bylevel': 0.90,
         'silent': True,
         'random_state': 42,
         'objective': 'reg:linear'}
pa_e0 = {'learning_rate': 0.05,      # Step size shrinkage used in update (Learning rate)
         'reg_alpha': 0.10,          # L1 regularization term on weights
         'n_estimators': 150,
         'max_depth': 4,
         'subsample': 1,
         'colsample_bytree': 0.90,
         'colsample_bylevel': 0.90,
         'silent': True,
         'random_state': 42,
         'objective': 'reg:linear'}

In [65]:
# Estimator pipeline
est_bg = Pipeline([
    ('scaler', StandardScaler()),
    ('xgbreg', XGBRegressor(**pa_bg)),
])
est_e0 = Pipeline([
    ('scaler', StandardScaler()),
    ('xgbreg', XGBRegressor(**pa_e0)),
])

In [66]:
est_bg.fit(X_train, y_bg_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.9, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=250,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
       reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

In [67]:
est_e0.fit(X_train, y_e0_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.9, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=150,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
       reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

### Output predictions

In [70]:
predicted_bg = est_bg.predict(X_test)
predicted_e0 = np.expm1(est_e0.predict(X_test))

In [71]:
predicted = pd.DataFrame({'formation_energy_ev_natom': predicted_e0,
                          'bandgap_energy_ev': predicted_bg}) \
              .reset_index().rename(columns={'index': 'id'})
predicted['id'] += 1

In [72]:
predicted.head()

,id,bandgap_energy_ev,formation_energy_ev_natom
0,1,1.555830,0.194493
1,2,3.653345,0.062524
2,3,3.428888,0.163906
3,4,3.070188,0.030007
4,5,1.563939,0.140292


In [73]:
err_e0 = rmsle(np.expm1(est_e0.predict(X_train)), np.expm1(y_e0_train))
err_bg = rmsle(est_bg.predict(X_train), y_bg_train)

In [74]:
# Training RMSLE values
print "RMSLE BG: {}, RMSLE E0: {}, RMSLE AVG: {}".format(err_bg, err_e0,
                                                         0.5 * (err_bg + err_e0))

RMSLE BG: 0.0728351173101, RMSLE E0: 0.027578048241, RMSLE AVG: 0.0502065827755


#### Write to file:

In [75]:
now = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
predicted.to_csv(os.path.join('output', 'xgb-ext-skl-{}.csv'.format(now)),
                columns=['id', 'formation_energy_ev_natom', 'bandgap_energy_ev'],
                index=False)